# Read real and generated datasets 

In [15]:
import scanpy as sc
from celldreamer.eval.compute_evaluation_metrics import process_labels, compute_evaluation_metrics
from scipy import sparse
import pandas as pd

In [16]:
def add_to_dict(d, metrics):
    for metric in metrics:
        if metric not in d:
            d[metric] = [metrics[metric]]
        else:
            d[metric]+=[metrics[metric]]
    return d

In [17]:
adata_real_path = "/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/processed_full_genome/c_elegans/c_elegans_test.h5ad"
adata_real = sc.read_h5ad(adata_real_path)
sc.pp.subsample(adata_real, n_obs=10000)

In [18]:
adata_real

AnnData object with n_obs × n_vars = 10000 × 17747
    obs: 'cell', 'numi', 'time_point', 'batch', 'size_factor', 'cell_type', 'cell_subtype', 'plot_cell_type', 'raw_embryo_time', 'embryo_time', 'embryo_time_bin', 'raw_embryo_time_bin', 'lineage', 'passed_qc'
    var: 'gene_id', 'gene_name', 'gene_description', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cell_type_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'X_counts'
    obsp: 'connectivities', 'distances'

In [ ]:
results_celldreamer = {}
results_scvi = {} 
results_scdiff = {}

for i in range(3):
    adata_generated_path_celldreamer = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/generated/neurips/generated_cells_{i}.h5ad"
    adata_generated_celldreamer = sc.read_h5ad(adata_generated_path_celldreamer)

    adata_generated_path_scvi = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/baseline_experiments/scvi/neurips/generated/neurips_{i}.h5ad"
    adata_generated_scvi = sc.read_h5ad(adata_generated_path_scvi)
    adata_generated_scvi.X = sparse.csr_matrix(adata_generated_scvi.X)
    adata_generated_scvi = process_labels(adata_real, adata_generated_scvi, "cell_type", categorical_obs=True)
    
    adata_generated_path_scDiffusion = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/baseline_experiments/scDiffusion/generated/neurips/generated_cells_{i}.h5ad"
    adata_generated_scDiffusion = sc.read_h5ad(adata_generated_path_scDiffusion)[-n_obs:]
    adata_generated_scDiffusion.X = sparse.csr_matrix(adata_generated_scDiffusion.X)
    adata_generated_scDiffusion = process_labels(adata_real, adata_generated_scDiffusion, "cell_type", categorical_obs=False)

    results_celldreamer_i = compute_evaluation_metrics(adata_real, adata_generated_celldreamer, "cell_type", conditional=True, nn=10, original_space=True)
    print("Finsihed celldreamer")
    results_scvi_i = compute_evaluation_metrics(adata_real, adata_generated_scvi, "cell_type", conditional=True, nn=10, original_space=True)
    print("Finsihed scvi")
    results_scdiff_i = compute_evaluation_metrics(adata_real, adata_generated_scDiffusion, "cell_type", conditional=True, nn=10, original_space=True)
    print("Finsihed scDiff")

    results_celldreamer = add_to_dict(results_celldreamer, results_celldreamer_i)
    results_scvi = add_to_dict(results_scvi, results_scvi_i)
    results_scdiff = add_to_dict(results_scdiff, results_scdiff_i)

/home/icb/alessandro.palma/environment/celldreamer/celldreamer/eval/compute_evaluation_metrics.py:21: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_generated.obs[category_field] = [labels_dict[int(lab)] for lab in np.array(adata_generated.obs[category_field])]
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/alessandro.palma/environment/celldreamer/celldreamer/eval/compute_evaluation_metrics.py:36: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initial

Finsihed celldreamer


/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/alessandro.palma/environment/celldreamer/celldreamer/eval/compute_evaluation_metrics.py:36: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata_generated.obsm["X_pca"] = adata_generated.X.A.dot(adata_real.varm["PCs"])


Finsihed scvi


/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca
/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/alessandro.palma/environment/celldreamer/celldreamer/eval/compute_evaluation_metrics.py:36: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata_generated.obsm["X_pca"] = adata_generated.X.A.dot(adata_real.varm["PCs"])


In [9]:
results_celldreamer_df = pd.DataFrame(results_celldreamer)
results_scvi_df = pd.DataFrame(results_scvi)
results_scdiff_df = pd.DataFrame(results_scdiff)

In [9]:
results_celldreamer_df.mean(0)

Exception ignored in: <function _xla_gc_callback at 0x7fb0c0eedea0>
Traceback (most recent call last):
  File "/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
KeyboardInterrupt: 


ERROR! Session/line number was not unique in

Exception ignored in: <function _xla_gc_callback at 0x7fb0c0eedea0>
Traceback (most recent call last):
  File "/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


 database. History logging moved to new session 8602


Exception ignored in: <function _xla_gc_callback at 0x7fb0c0eedea0>
Traceback (most recent call last):
  File "/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
results_scvi_df.mean(0)

In [ ]:
results_scdiff_df.mean(0)

In [13]:
results_celldreamer_i

{'1-Wasserstein_PCA': 20.333249217255783,
 '2-Wasserstein_PCA': 20.544687057269222,
 'Linear_MMD_PCA': 373.039306640625,
 'Poly_MMD_PCA': 144222.53125,
 '1-Wasserstein': 23.65248480024329,
 '2-Wasserstein': 23.87172915841405,
 'Linear_MMD': 122.11853790283203,
 'Poly_MMD': 98860.9921875,
 'KNN identity': 0.005222222222222222,
 'KNN identity PCA': 5.555555555555556e-05,
 'precision': 0.07666666666666666,
 'recall': 0.3387,
 'density': 0.055477777777777786,
 'coverage': 0.2824,
 'precision_PCA': 0.0017222222222222222,
 'recall_PCA': 0.2994,
 'density_PCA': 0.00028333333333333335,
 'coverage_PCA': 0.0024,
 'KNN category': 0.6926111111111111,
 'KNN category PCA': 0.5777222222222222}

In [14]:
results_scvi_i

{'1-Wasserstein_PCA': 21.054406014017424,
 '2-Wasserstein_PCA': 21.278284015130694,
 'Linear_MMD_PCA': 396.2701416015625,
 'Poly_MMD_PCA': 164699.625,
 '1-Wasserstein': 24.338574693079142,
 '2-Wasserstein': 24.564836063482858,
 'Linear_MMD': 148.68447875976562,
 'Poly_MMD': 124298.7265625,
 'KNN identity': 0.005096105910375007,
 'KNN identity PCA': 5.5392455547554426e-05,
 'precision': 0.06021159918019166,
 'recall': 0.3519,
 'density': 0.08468952528665596,
 'coverage': 0.2352,
 'precision_PCA': 0.0013848113886888605,
 'recall_PCA': 0.0016,
 'density_PCA': 0.00017725585775217416,
 'coverage_PCA': 0.0011,
 'KNN category': 0.6191214756550157,
 'KNN category PCA': 0.5203013349581787}